In [1]:
import os
os.environ["RANK"] = "0"
os.environ["WORLD_SIZE"] = "1"
os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = "12500"
import torch
import torch.distributed as dist

torch.cuda.set_device(0)
dist.init_process_group(backend='nccl')
config_list = [
    "assets/HLLM/HLLM.yaml",
    "assets/overall/LLM_deepspeed.yaml"
]
from config.configurator import Config
cfg = Config(config_list)
print(cfg)
# torch.manual_seed(cfg.get("seed"))



/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.



General Hyper Parameters:
model = HLLM
seed = 2020
state = INFO
use_text = True
reproducibility = True
checkpoint_dir = saved
show_progress = True
log_wandb = False
data_path = /mnt/workspace/dataset/hllm/dataset
strategy = deepspeed
precision = bf16-mixed

Training Hyper Parameters:
epochs = 10
train_batch_size = 2
optim_args = {'learning_rate': 0.0001, 'weight_decay': 0.01}
eval_step = 1
stopping_step = 5

Evaluation Hyper Parameters:
eval_batch_size = 256
topk = [5, 10, 50, 200]
metrics = ['Recall', 'NDCG']
valid_metric = NDCG@200
metric_decimal_place = 7
eval_type = EvaluatorType.RANKING
valid_metric_bigger = True

Dataset Hyper Parameters:
MAX_ITEM_LIST_LENGTH = 25
MAX_TEXT_LENGTH = 32
text_keys = ['title', 'tag', 'description']
item_prompt = Compress the following sentence into embedding: 

Other Hyper Parameters: 
item_pretrain_dir = /mnt/workspace/model/TinyLlama-1.1B
item_llm_init = True
user_pretrain_dir = /mnt/workspace/model/TinyLlama-1.1B
user_llm_init = True
use_ft_flash

In [2]:
local_rank = 0
device = torch.device("cuda", local_rank)
cfg['device'] = device
extra_args_str = \
"""
--loss nce
--epochs 5
--dataset Pixel200K
--train_batch_size 16
--MAX_TEXT_LENGTH 256
--MAX_ITEM_LIST_LENGTH 10
--checkpoint_dir saved_path
--optim_args.learning_rate 1e-4
--item_pretrain_dir item_pretrain_dir
--user_pretrain_dir user_pretrain_dir
--text_path text_path
--text_keys [\"title\", \"tag\", \"description\"]
"""
extra_args_list = extra_args_str.strip().split("\n")
extra_args_list = [args.split(" ", maxsplit=1) for args in extra_args_list]
extra_args = []
for args_list in extra_args_list:
    extra_args.extend(args_list)
# cfg.parse_extra_args(extra_args)

from utils import init_seed
init_seed(cfg['seed'], cfg['reproducibility'])

# dist.destroy_process_group()

In [3]:
from data import load_data

dataload = load_data(cfg)

from data import bulid_dataloader

train_loader, valid_loader, test_loader = bulid_dataloader(cfg, dataload)

print(dataload)

2020
Pixel200K
The number of users: 200001
Average actions of users: 19.82828
The number of items: 96283
Average actions of items: 41.187927130720176
The number of inters: 3965656
The sparsity of the dataset: 99.9794063532928%


In [4]:
it = iter(train_loader)
first_bt = next(it)
print(first_bt)

/mnt/workspace/code/RecomAlgor_torch/data/dataset/collate_fn.py:41: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  storage = elem.storage()._new_shared(numel)
/mnt/workspace/code/RecomAlgor_torch/data/dataset/collate_fn.py:41: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  storage = elem.storage()._new_shared(numel)
/mnt/workspace/code/RecomAlgor_torch/data/dataset/collate_fn.py:41: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This shou

{'pos_item_ids': tensor([[15044, 13347,  4909, 71459, 39487, 19671, 42188,  9202, 20558,  6537,
          2267,  3277, 37859,  8100, 28494, 22934,  1195, 37891,  7609, 44894,
          5914, 28193, 17560, 21251, 15377, 17749],
        [ 1323,  7260, 72237,  2663, 51230,  2435,  8988, 14559, 15774, 60457,
         44285, 51112, 27182,  3003,  7348, 10434,  7605, 10342, 45941, 12914,
         14676,   119,  5856, 11143,  4696,  8186]]), 'neg_item_ids': tensor([[87832, 81222, 81198, 22876, 87871, 60427, 57998, 62150, 48319, 54961,
         71171, 20766, 80933, 12016, 60008, 31466, 27471, 53705, 87849, 26910,
         79237, 19517, 61800, 59291, 13761, 92493],
        [84049, 21196, 64375, 81476, 70445, 62330,  5355, 58035,  6468, 45213,
         28445, 15939, 69716, 90774, 12262, 80347, 34469, 20374,  3443, 84371,
         70657, 39371, 60806, 31117, 40260, 65725]]), 'pos_input_ids': tensor([    1,   422,  2139,  ...,  8216, 29901,     0]), 'pos_cu_input_lens': tensor([33, 33, 33, 33, 33,

In [5]:
import logging as log
from utils.logger import init_logger
init_logger(cfg)
logger = log.getLogger()

/usr/local/lib/python3.11/site-packages/torch/distributed/distributed_c10d.py:4876: UserWarning: barrier(): using the device under current context. You can specify `device_id` in `init_process_group` to mute this warning.
  warnings.warn(  # warn only once
[rank0]:[W201 21:38:38.982665686 ProcessGroupNCCL.cpp:5072] Guessing device ID based on global rank. This can cause a hang if rank to GPU mapping is heterogeneous. You can specify device_id in init_process_group()


In [6]:
from utils import get_model
model = get_model(cfg['model'])(cfg, dataload)

01 Feb 21:38    INFO  create item llm
01 Feb 21:38    INFO  ******* create LLM /mnt/workspace/model/TinyLlama-1.1B *******
01 Feb 21:38    INFO  hf_config: LlamaConfig {
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "dtype": "float32",
  "eos_token_id": 2,
  "head_dim": 64,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 5632,
  "max_position_embeddings": 2048,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 22,
  "num_key_value_heads": 4,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "transformers_version": "4.57.6",
  "use_cache": false,
  "vocab_size": 32001
}

01 Feb 21:38    INFO  xxxxx starting loading checkpoint
01 Feb 21:38    INFO  Using flash attention True for llama
01 Feb 21:38    INFO  Init True for llama


LlamaForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly defined. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
LlamaForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly defined. However, it doesn't directly inherit from `GenerationMixin`. Fr

01 Feb 21:38    INFO  create user llm
01 Feb 21:38    INFO  ******* create LLM /mnt/workspace/model/TinyLlama-1.1B *******
01 Feb 21:38    INFO  hf_config: LlamaConfig {
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "dtype": "float32",
  "eos_token_id": 2,
  "head_dim": 64,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 5632,
  "max_position_embeddings": 2048,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 22,
  "num_key_value_heads": 4,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "transformers_version": "4.57.6",
  "use_cache": false,
  "vocab_size": 32001
}

01 Feb 21:38    INFO  xxxxx starting loading checkpoint
01 Feb 21:38    INFO  Using flash attention True for llama
01 Feb 21:38    INFO  Init True for llama


LlamaForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly defined. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
LlamaForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly defined. However, it doesn't directly inherit from `GenerationMixin`. Fr

01 Feb 21:38    INFO  nce thres setting to 0.99


In [7]:
interaction = first_bt
user_attention_mask = interaction['attention_mask']
N, S = user_attention_mask.shape
model = model.type(torch.float16).cuda()
for key, tensor in interaction.items():
    interaction[key] = tensor.cuda()
pos_input_ids, pos_cu_input_lens, pos_position_ids = interaction['pos_input_ids'], interaction['pos_cu_input_lens'], interaction['pos_position_ids']
neg_input_ids, neg_cu_input_lens, neg_position_ids = interaction['neg_input_ids'], interaction['neg_cu_input_lens'], interaction['neg_position_ids']
pos_embedding = model.forward_item_emb(pos_input_ids, pos_position_ids, pos_cu_input_lens, model.item_emb_token_n, model.item_emb_tokens, model.item_llm)
pos_embedding = pos_embedding.reshape(N, S+1, -1)

In [8]:
print(neg_input_ids.shape)
print(neg_position_ids.shape)
print(neg_cu_input_lens.shape)
print(model.num_negatives)

torch.Size([1701])
torch.Size([1701])
torch.Size([52])
None


In [9]:
neg_embedding = model.forward_item_emb(neg_input_ids, neg_position_ids, neg_cu_input_lens, model.item_emb_token_n, model.item_emb_tokens, model.item_llm)
neg_embedding = neg_embedding.reshape(N, -1, model.item_llm.config.hidden_size)
print(neg_embedding.shape)


torch.Size([2, 26, 2048])


In [10]:
from trainer.trainer import Trainer
world_size = torch.distributed.get_world_size()
trainer = Trainer(cfg, model)

df: /root/.triton/autotune: 没有那个文件或目录


ModuleNotFoundError: No module named 'lightning'